<a href="https://colab.research.google.com/github/yasirabd/solver-society-job-analysis/blob/main/6_Preprocessing_%26_Feature_Engineering_Jobstreet_(New).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
- Cleaning data
- Feature engineering
- Export

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Data Loker/(raw) jobstreet_25jan_24feb_2021.csv")
df.tail()

,platform,job_id,posted_date,closing_date,job_position,company_name,salary,company_location,avg_process_time,telephone_number,working_hours,company_website,company_size,dress_code,nearby_locations,company_overview,job_description,career_level,years_of_experience,qualification,field_of_study,industry,skills,employment_type,languages,job_function,benefits
20896,jobstreet,3465849,2021-02-24 06:35:40,2021-03-26 06:35:40,Operations Manager - Logistics for Mining - Su...,PT Michael Page Internasional Indonesia,IDR 25000000 - 35000000 / monthly,Sulawesi Tenggara,30 days,NaN,NaN,NaN,51 - 200 pekerja,NaN,NaN,"<div style=""text-align:justify"">Michael Page I...",<div>The position targets to lead the site man...,Manajer/Asisten Manajer,10 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",NaN,Manajemen/Konsulting HR,NaN,Penuh Waktu,NaN,"Pelayanan,Logistik/Rantai Pasokan",NaN
20897,jobstreet,3465850,2021-02-24 06:39:52,2021-03-26 06:39:52,Associate or Senior Associate - Venture Capital,PT Michael Page Internasional Indonesia,IDR 40000000 - 70000000 / monthly,Jakarta Raya,30 days,NaN,NaN,NaN,51 - 200 pekerja,NaN,NaN,"<div style=""text-align:justify"">Michael Page I...",<div>Our client is one of the r<strong>eputabl...,Manajer/Asisten Manajer,3 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",NaN,Manajemen/Konsulting HR,NaN,Penuh Waktu,NaN,"Akuntansi / Keuangan,Keuangan / Investasi Peru...",NaN
20898,jobstreet,3465851,2021-02-24 06:47:46,2021-03-26 06:47:46,Tech Architect - Unicorn Tech Company,PT Michael Page Internasional Indonesia,IDR 35000000 - 50000000 / monthly,Jakarta Raya,30 days,NaN,NaN,NaN,51 - 200 pekerja,NaN,NaN,"<div style=""text-align:justify"">Michael Page I...",<div>As a Tech Architect you will work on an e...,Manajer/Asisten Manajer,8 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",NaN,Manajemen/Konsulting HR,NaN,Penuh Waktu,NaN,"Komputer/Teknologi Informasi,IT-Perangkat Lunak",NaN
20899,jobstreet,3465853,2021-02-24 07:07:07,2021-03-26 07:07:07,Senior Data Scientist (Machine Learning),PT Michael Page Internasional Indonesia,IDR 30000000 - 40000000 / monthly,Jakarta Raya,30 days,NaN,NaN,NaN,51 - 200 pekerja,NaN,NaN,"<div style=""text-align:justify"">Michael Page I...",<div>You will be independently determining the...,Supervisor/Koordinator,3 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",NaN,Manajemen/Konsulting HR,NaN,Penuh Waktu,NaN,"Sains,Aktuaria/Statistik",NaN
20900,jobstreet,3465854,2021-02-24 07:07:53,2021-03-26 07:07:53,Accounting Officer,PT Suprabakti Mandiri,IDR 5000000 - 6500000 / monthly,Jakarta Utara,29 days,NaN,NaN,NaN,501 - 1000 pekerja,NaN,Sunter,"<div style=""text-align:justify""><strong>PENGUM...",<div>Tugas dan Tanggung Jawab:</div><ul><li>Fi...,Pegawai (non-manajemen & non-supervisor),2 tahun,Sarjana (S1),NaN,Konstruksi/Bangunan/Teknik,NaN,Penuh Waktu,NaN,"Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan","Tip,Asuransi kesehatan,Waktu regular, Senin - ..."


In [3]:
# import dataset
df = pd.read_csv("/content/drive/MyDrive/Data Loker/(raw) jobstreet_25jan_24feb_2021.csv")

# drop some attributes
drop_cols = ['job_id', 'telephone_number', 'working_hours', 'company_website', 'dress_code',
             'nearby_locations', 'company_overview', 'field_of_study', 'skills', 'languages',
             'posted_date', 'closing_date', 'job_description']  # add job_description, karena belum bisa ekstrak skill
df = df.drop(drop_cols, axis=1)
df.head()

,platform,job_position,company_name,salary,company_location,avg_process_time,company_size,career_level,years_of_experience,qualification,industry,employment_type,job_function,benefits
0,jobstreet,Staff IT,PT Nichirin Indonesia,IDR 4900000 - 5800000 / monthly,Karawang,28 days,51 - 200 pekerja,Pegawai (non-manajemen & non-supervisor),1 tahun,Sarjana (S1),Automobil/Mesin Tambahan Automotif/Kendaraan,Penuh Waktu,"Komputer/Teknologi Informasi,IT-Admin Jaringan...","Asuransi kesehatan,Waktu regular, Senin - Juma..."
1,jobstreet,Management Trainee,PT Sulfindo Adiusaha,IDR 6500000 - 8500000 / monthly,Serang,28 days,201 - 500 pekerja,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,NaN,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Bahan Kimia/Pupuk/Pestisida,Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...","Tip,Waktu regular, Senin - Jumat,Bisnis (conto..."
2,jobstreet,Admin Spare Part (Warehouse),WINGS Group,IDR 4800000 - 5000000 / monthly,Jakarta Timur,19 days,Lebih dari 5000 pekerja,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Produk Konsumen/Barang konsumen yang bergerak ...,Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material","Asuransi kesehatan,Bisnis (contoh: Kemeja),Ope..."
3,jobstreet,Kepala Cabang Bandar Lampung,PT Puji Lestari Purnama,IDR 9000000 - 15000000 / monthly,Bandar Lampung,26 days,1- 50 pekerja,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),Umum & Grosir,Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...","Asuransi kesehatan,Waktu regular, Senin - Juma..."
4,jobstreet,Field Operation Engineer (Telecommunication) -...,FIBERSTAR (PT. Mega Akses Persada),IDR 4000000 - 7000000 / monthly,Jakarta Selatan,22 days,201 - 500 pekerja,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Telekomunikasi,Penuh Waktu,"Teknik,Teknik Elektronika","Asuransi Gigi,Tip,Asuransi kesehatan,Parkir,Pe..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20901 entries, 0 to 20900
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   platform             20901 non-null  object
 1   job_position         20901 non-null  object
 2   company_name         20160 non-null  object
 3   salary               20901 non-null  object
 4   company_location     20901 non-null  object
 5   avg_process_time     17240 non-null  object
 6   company_size         19245 non-null  object
 7   career_level         20901 non-null  object
 8   years_of_experience  19638 non-null  object
 9   qualification        20901 non-null  object
 10  industry             20160 non-null  object
 11  employment_type      20901 non-null  object
 12  job_function         20901 non-null  object
 13  benefits             18295 non-null  object
dtypes: object(14)
memory usage: 2.2+ MB


In [5]:
df.describe().T

,count,unique,top,freq
platform,20901,1,jobstreet,20901
job_position,20901,14961,Sales Executive,118
company_name,20160,6804,ACT Foundation,196
salary,20901,1757,IDR 5000000 - 6500000 / monthly,860
company_location,20901,193,Jakarta Raya,4131
avg_process_time,17240,30,29 days,3339
company_size,19245,7,51 - 200 pekerja,5192
career_level,20901,6,Pegawai (non-manajemen & non-supervisor),11484
years_of_experience,19638,18,1 tahun,6276
qualification,20901,36,Sarjana (S1),9842


# Data Cleaning

In [6]:
data = df.copy()

In [7]:
import re

# drop duplicate
def drop_duplicate(df):
    return df.drop_duplicates().reset_index(drop=True)

def clean_platform(platform):
    return platform.str.capitalize()

def extract_job_position_by_keyword(job_l, name_job, keywords):
    for i,j in enumerate(job_l):
        if name_job in j:
            clean_job = []
            for item in j.split():
                if item in keywords: clean_job.append(item)
            job_l[i] = name_job if len(clean_job)<1 else ' '.join(clean_job)
    return job_l

def extract_job_position_by_freq(job_series, freq=3):
    top_job = list(job_series.value_counts()[job_series.value_counts()>freq].index)
    temp = []
    for t in top_job:
        temp += t.split()
    keywords = list(set(temp))

    job_l = job_series.to_list()
    for i,j in enumerate(job_l):
        clean_job = []
        for item in j.split():
            if item in keywords: clean_job.append(item)
            job_l[i] = job_l[i] if len(clean_job)<1 else ' '.join(clean_job)
    return job_l

def remove_job_extra(job_l):
    # remove data in brackets
    brackets = ['(', ')', '[', ']']
    for idx,job in enumerate(job_l):
        for b in brackets:
            if b in job: job = re.sub("[\(\[].*?[\)\]]", "", job)
    
        # remove penjelas after '-'
        if '-' in job: job = job.split('-')[0].strip()

        # remove penjelas after ':'
        if ':' in job: job = job.split(':')[0].strip()

        job_l[idx] = job
    return job_l

def clean_job_position(job_post):
    # lowercase
    job_post = job_post.str.lower()
    job_l = job_post.to_list()

    # extract by keywords
    sales_keywords = ['sales', 'executive', 'staff', 'marketing', 'supervisor',
                      'manager', 'engineer', 'officer', 'representative']
    accounting_keywords = ['accounting', 'finance', 'staff', 'manager', 'tax',
                           'supervisor', 'assistant']
    marketing_keywords = ['marketing', 'sales', 'digital', 'manager', 'staff',
                          'executive', 'communication', 'supervisor']
    hrd_keywords = ['hrd', 'manager', 'staff', 'ga', 'supervisor', '&']
    programmer_keywords = ['programmer', 'it', 'web', 'developer', 'android',
                           'php', 'mobile', 'java']
    
    # extract job by keyword
    job_l = extract_job_position_by_keyword(job_l, 'sales', sales_keywords)
    job_l = extract_job_position_by_keyword(job_l, 'accounting', accounting_keywords)
    job_l = extract_job_position_by_keyword(job_l, 'marketing', marketing_keywords)
    job_l = extract_job_position_by_keyword(job_l, 'hrd', hrd_keywords)
    job_l = extract_job_position_by_keyword(job_l, 'programmer', programmer_keywords)

    # extract job by frequency
    job_l = extract_job_position_by_freq(pd.Series(job_l), freq=8)

    # remove extra word in job position, such as
    # '(jakarta)' in 'sales executive (jakarta)'
    # '- jakarta' in 'sales executive - jakarta'
    # ': jakarta' in 'sales executive: jakarta'
    job_l = remove_job_extra(job_l)

    # capitalize first word
    job_l = [j.title() for j in job_l]
    return job_l

def clean_company_name(company_name):
    # fillna with 'Tidak Dicantumkan'
    return company_name.fillna('Tidak Dicantumkan')

def clean_salary(salary):
    # remove '/ monthly'
    for idx,sal in enumerate(salary):
        sal = sal.split('/')[0].strip()

        # give decimal in currency
        sal = sal.split()
        temp = []
        for s in sal:
            if s.isnumeric(): 
                temp.append('{:15,.0f}'.format(int(s)).strip())
            else:
                temp.append(s.strip())
        salary[idx] = ' '.join(temp)
    return salary

def clean_avg_process_time(process_time):
    # fillna with maximum process time = 30
    process_time = process_time.fillna('30')

    # remove 'day' or 'days
    for idx,t in enumerate(process_time):
        process_time[idx] = int(t.split()[0])
    return process_time

def clean_company_size(company_size):
    return company_size.fillna('Tidak dicantumkan')

def clean_years_of_experience(yoe):
    # fill na with '0'
    yoe = yoe.fillna('0')

    # replace 'Lebih dari 20 Tahun' into '20'
    yoe = yoe.replace('Lebih dari 20 Tahun', '20')

    # remove 'tahun'
    for idx,y in enumerate(yoe):
        yoe[idx] = int(y.split()[0])
    return yoe

def clean_industry(industry):
    return industry.fillna('Lainnya')

def clean_job_function(job_function):
    for idx,func in enumerate(job_function):
        split_func = func.split(',')
        join_func = ', '.join(split_func)
        split_func = [j.strip() for j in join_func.split('/')]

        job_function[idx] = '/'.join(split_func)
    return job_function

def extract_benefit_from_keywords(benefit_l, keywords):

    for i,b in enumerate(benefit_l):
        b_split = [res.strip() for res in b.split(',')]
        temp = []
        for bs in b_split:
            if bs in keywords:
                temp.append(bs)
        benefit_l[i] = 'tidak dicantumkan' if len(temp)<1 else ', '.join(temp)
    return benefit_l

def mapping_benefit(benefit_l, mappers):
    
    for i,b in enumerate(benefit_l):
        b_split = [res.strip() for res in b.split(',')]

        # mapping benefit
        mapped = pd.Series(b_split).map(mappers).dropna().drop_duplicates().tolist()
        mapped = [m.title() for m in mapped]  # capitalize first character each word
        benefit_l[i] = 'Tidak Dicantumkan' if len(mapped)<1 else ', '.join(mapped)
    
    return benefit_l

def clean_benefits(benefits):
    # lowercase
    benefits = benefits.str.lower()
    # fillna with 'tidak dicantumkan'
    benefits = benefits.fillna('tidak dicantumkan')
    benefit_l = benefits.to_list()

    # extract from keywords
    keywords = ['asuransi','kesehatan','tip','parkir','gigi','penglihatan','tunjangan',
                'pendidikan','bpjs','ketenagakerjaan','thr','bonus','jamsostek','transport',
                'lunch','insurance','insentif','meal','incentive','transportation', 'makan',
                'komisi','transportasi','medical','pulsa','commission','maternity','tempat',
                'pension','bpjstk','kacamata','bonuses','dormitory','parking','pensiun',
                'parking','reward','jaminan','incentives','education','insentif','insentive',
                'health','birth','inssurance','umroh','haji','commissions','glasses','travel',
                'optics','trips','trip','commission','tiket','glasses','meals','car','motorcycle',
                'gasoline','mess','insentive','jht','retirement','commission']
    # extract benefit from list of keyword
    benefit_l = extract_benefit_from_keywords(benefit_l, keywords)
    # mapping benefit
    mappers = {'asuransi': 'asuransi',
               'kesehatan': 'kesehatan',
               'tip': 'bonus',
               'parkir': 'parkir',
               'gigi': 'gigi',
               'penglihatan': 'penglihatan',
               'tunjangan': 'tunjangan',
               'pendidikan': 'pendidikan',
               'bpjs': 'bpjs',
               'ketenagakerjaan': 'ketenagakerjaan',
               'thr': 'tunjangan',
               'bonus': 'bonus',
               'jamsostek': 'bpjs',
               'transport': 'transportasi',
               'lunch': 'makan siang',
               'insurance': 'asuransi',
               'insentif': 'bonus',
               'meal': 'makan siang',
               'incentive': 'bonus',
               'transportation': 'transportasi',
               'makan': 'makan siang',
               'komisi': 'bonus',
               'transportasi': 'transportasi',
               'medical': 'kesehatan',
               'pulsa': 'pulsa',
               'commission': 'bonus',
               'maternity': 'cuti hamil',
               'tempat': 'tempat tinggal',
               'pension': 'tunjangan pensiun',
               'bpjstk': 'bpjs ketenagakerjaan',
               'kacamata': 'penglihatan',
               'bonuses': 'bonus',
               'dormitory': 'tempat tinggal',
               'parking': 'parkir',
               'pensiun': 'tunjangan pensiun',
               'parking': 'parkir',
               'reward': 'bonus',
               'jaminan': 'jaminan',
               'incentives': 'bonus',
               'education': 'pendidikan',
               'insentif': 'bonus',
               'insentive': 'bonus',
               'health': 'kesehatan',
               'birth': 'cuti hamil',
               'inssurance': 'asuransi',
               'umroh': 'umroh',
               'haji': 'haji',
               'commissions': 'bonus',
               'glasses': 'penglihatan',
               'travel': 'perjalanan kerja',
               'optics': 'penglihatan',
               'trips': 'perjalanan kerja',
               'trip': 'perjalanan kerja',
               'commission': 'bonus',
               'tiket': 'perjalanan kerja',
               'glasses': 'penglihatan',
               'meals': 'makan siang',
               'car': 'transportasi',
               'motorcycle': 'transportasi',
               'gasoline': 'transportasi',
               'mess': 'tempat tinggal',
               'insentive': 'bonus',
               'jht': 'jaminan hari tua',
               'retirement': 'tunjangan pensiun',
               'commission': 'bonus',
               'tidak dicantumkan': 'tidak dicantumkan'}
    benefit_l = mapping_benefit(benefit_l, mappers)
    return benefit_l

In [8]:
def preprocess_data(df):
    df = drop_duplicate(df)
    df['platform'] = clean_platform(df.platform)  # 1: platform
    df['job_position'] = clean_job_position(df.job_position)   # 2: job_position
    df['company_name'] = clean_company_name(df.company_name)  # 3: company_name
    df['salary'] = clean_salary(df.salary)  # 4: salary
    # 5: company_location ==========> Bagian Feature Engineering
    df['avg_process_time'] = clean_avg_process_time(df.avg_process_time)  # 6: avg_process_time
    df['company_size'] = clean_company_size(df.company_size)  # 7: company_size
    # 8: job_description ==========> Tidak digunakan
    # 9: career_level -> tidak dilakukan cleaning
    df['years_of_experience'] = clean_years_of_experience(df.years_of_experience)  # 10: years_of_experience

    # 11: qualification -> tidak dilakukan cleaning
    # Gelar: 'D3 (Diploma)', 'D4 (Diploma)', 'Diploma Pascasarjana', 'Doktor (S3)',
    #        'Gelar Professional', 'Magister (S2)', 'SMA', 'SMU/SMK/STM',
    #        'Sarjana (S1)', 'Sertifikat Professional', 'Tidak terspesifikasi'

    df['industry'] = clean_industry(df.industry)  # 12: industry

    # 13: employment_type -> tidak dilakukan cleaning
    # Employment Type: 'Kontrak', 'Magang', 'Paruh Waktu', 'Penuh Waktu', 'Temporer'

    df['job_function'] = clean_job_function(df.job_function)  # 14: job_function

    df['benefits'] = clean_benefits(df.benefits)  # 15: benefits
    # Extract benefits into this items:
    # 'bonus', 'parkir', 'penglihatan', 'makan siang', 'bpjs'
    # 'tunjangan', 'transportasi', 'asuransi', 'kesehatan', 'pulsa',
    # 'tempat tinggal', 'tunjangan pensiun', 'cuti hamil', 'ketenagakerjaan', 'jaminan hari tua',
    # 'perjalanan kerja', 'bpjs ketenagakerjaan', 'umroh', 'haji', 'gigi',
    # 'pendidikan', 'tidak dicantumkan'

    return df

In [9]:
data_preprocessed = preprocess_data(data)
data_preprocessed.head()

,platform,job_position,company_name,salary,company_location,avg_process_time,company_size,career_level,years_of_experience,qualification,industry,employment_type,job_function,benefits
0,Jobstreet,Staff It,PT Nichirin Indonesia,"IDR 4,900,000 - 5,800,000",Karawang,28,51 - 200 pekerja,Pegawai (non-manajemen & non-supervisor),1,Sarjana (S1),Automobil/Mesin Tambahan Automotif/Kendaraan,Penuh Waktu,"Komputer/Teknologi Informasi, IT-Admin Jaringa...",Tidak Dicantumkan
1,Jobstreet,Management Trainee,PT Sulfindo Adiusaha,"IDR 6,500,000 - 8,500,000",Serang,28,201 - 500 pekerja,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,0,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Bahan Kimia/Pupuk/Pestisida,Penuh Waktu,"Penjualan/Pemasaran, Pemasaran/Pengembangan Bi...",Bonus
2,Jobstreet,Admin,WINGS Group,"IDR 4,800,000 - 5,000,000",Jakarta Timur,19,Lebih dari 5000 pekerja,Pegawai (non-manajemen & non-supervisor),1,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Produk Konsumen/Barang konsumen yang bergerak ...,Penuh Waktu,"Manufaktur, Pembelian/Manajemen Material",Makan Siang
3,Jobstreet,Kepala Cabang,PT Puji Lestari Purnama,"IDR 9,000,000 - 15,000,000",Bandar Lampung,26,1- 50 pekerja,Manajer/Asisten Manajer,5,Sarjana (S1),Umum & Grosir,Penuh Waktu,"Penjualan/Pemasaran, Pemasaran/Pengembangan Bi...",Tidak Dicantumkan
4,Jobstreet,Operation Engineer,FIBERSTAR (PT. Mega Akses Persada),"IDR 4,000,000 - 7,000,000",Jakarta Selatan,22,201 - 500 pekerja,Pegawai (non-manajemen & non-supervisor),2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Telekomunikasi,Penuh Waktu,"Teknik, Teknik Elektronika","Bonus, Parkir, Penglihatan, Bpjs"


# Feature Engineering
Ekstrak company_location menjadi:
- pulau
- provinsi

In [10]:
def normalize_location(location):
    loc = location.copy()
    loc = loc.replace('Jakarta Raya', 'DKI Jakarta')
    loc = loc.replace('Jakarta Pusat', 'Kota Administrasi Jakarta Pusat')
    loc = loc.replace('Jakarta Selatan', 'Kota Administrasi Jakarta Selatan')
    loc = loc.replace('Jakarta Barat', 'Kota Administrasi Jakarta Barat')
    loc = loc.replace('Jakarta Utara', 'Kota Administrasi Jakarta Utara')
    loc = loc.replace('Jakarta Timur', 'Kota Administrasi Jakarta Timur')
    loc = loc.replace('Surabaya', 'Kota Surabaya')
    loc = loc.replace('Medan', 'Kota Medan')
    loc = loc.replace('Yogyakarta', 'DI Yogyakarta')
    loc = loc.replace('Palembang', 'Kota Palembang')
    loc = loc.replace('Depok', 'Kota Depok')
    loc = loc.replace('Makassar', 'Kota Makassar')
    loc = loc.replace('Batam', 'Kota Batam')
    loc = loc.replace('Surakarta', 'Kota Surakarta')
    loc = loc.replace('Denpasar', 'Kota Denpasar')
    loc = loc.replace('Malang', 'Kota Malang')
    loc = loc.replace('Serang', 'Kota Serang')
    loc = loc.replace('Balikpapan', 'Kota Balikpapan')
    loc = loc.replace('Pekanbaru', 'Kota Pekanbaru')
    loc = loc.replace('Banjarmasin', 'Kota Banjarmasin')
    loc = loc.replace('Samarinda', 'Kota Samarinda')
    loc = loc.replace('Cilegon', 'Kota Cilegon')
    loc = loc.replace('Manado', 'Kota Manado')
    loc = loc.replace('Pontianak', 'Kota Pontianak')
    loc = loc.replace('Cileungsi', 'Bogor')
    loc = loc.replace('Pontianak', 'Kota Pontianak')
    loc = loc.replace('Cimahi', 'Kota Cimahi')
    loc = loc.replace('Jambi', 'Kota Jambi')
    loc = loc.replace('Padang', 'Kota Padang')
    loc = loc.replace('Palu', 'Kota Palu')
    loc = loc.replace('Purwokerto', 'Purwakarta')
    loc = loc.replace('Kendari', 'Kota Kendari')
    loc = loc.replace('Halmahera', 'Maluku Utara')
    loc = loc.replace('Salatiga', 'Kota Salatiga')
    loc = loc.replace('Lombok', 'Nusa Tenggara Barat')
    loc = loc.replace('Citeureup', 'Bogor')
    loc = loc.replace('Bengkulu', 'Kota Bengkulu')
    loc = loc.replace('Magelang', 'Kota Magelang')
    loc = loc.replace('Ambon', 'Kota Ambon')
    loc = loc.replace('Palangkaraya', 'Kota Palangka Raya')
    loc = loc.replace('Bandar Lampung', 'Kota Bandar Lampung')
    loc = loc.replace('Ternate', 'Kota Ternate')
    loc = loc.replace('Rangkasbitung', 'Lebak')
    loc = loc.replace('Timika', 'Papua')
    loc = loc.replace('Dumai', 'Kota Dumai')
    loc = loc.replace('Cibinong', 'Bogor')
    loc = loc.replace('Madura', 'Bangkalan')
    loc = loc.replace('Prabumulih', 'Kota Prabumulih')
    loc = loc.replace('Mataram', 'Kota Mataram')
    loc = loc.replace('Tanjung Pinang', 'Kota Tanjungpinang')
    loc = loc.replace('Bontang', 'Kota Bontang')
    loc = loc.replace('Madiun', 'Kota Madiun')
    loc = loc.replace('Probolinggo', 'Kota Probolinggo')
    loc = loc.replace('Bangka Belitung', 'Kepulauan Bangka Belitung')
    loc = loc.replace('Pangkal Pinang', 'Kota Pangkalpinang')
    loc = loc.replace('Banjarbaru', 'Kota Banjarbaru')
    loc = loc.replace('Padang Sidempuan', 'Kota Padang')
    loc = loc.replace('Seminyak', 'Bali')
    loc = loc.replace('Pematangsiantar', 'Kota Pematangsiantar')
    loc = loc.replace('Binjai', 'Kota Binjai')
    loc = loc.replace('Ungaran', 'Semarang')
    loc = loc.replace('Tapanuli', 'Sumatera Utara')
    loc = loc.replace('Sibolga', 'Kota Sibolga')
    loc = loc.replace('Meulaboh', 'Aceh')
    loc = loc.replace('Tarakan', 'Kota Tarakan')
    loc = loc.replace('Ubud', 'Bali')
    loc = loc.replace('Kuta', 'Bali')
    loc = loc.replace('Singkawang', 'Kota Singkawang')
    loc = loc.replace('Batu', 'Kota Batu')
    loc = loc.replace('Bukittinggi', 'Kota Bukittinggi')
    loc = loc.replace('Tomohon', 'Kota Tomohon')
    loc = loc.replace('Kotamobagu', 'Kota Kotamobagu')
    loc = loc.replace('Gunung Kidul', 'Gunungkidul')
    loc = loc.replace('Cikarang', 'Bekasi')
    loc = loc.replace('Takengon', 'Aceh')
    loc = loc.replace('Metro', 'Kota Metro')
    loc = loc.replace('Pare-Pare', 'Kota Parepare')
    loc = loc.replace('Kabupaten Katingan', 'Katingan')
    loc = loc.replace('Asahan Kisaran', 'Sumatera Utara')
    loc = loc.replace('Toraja', 'Sulawesi Selatan')
    return loc

def extract_location(location):
    # load daftar lokasi indonesia berdasarkan pulau, provinsi, kabupaten/kota
    loc_vocab = pd.read_csv("/content/drive/My Drive/Data Loker/daftar_kota_kabupaten_id.csv")
    kab_kota_l = loc_vocab['Kabupaten/Kota'].tolist()
    prov_l = loc_vocab['Provinsi'].tolist()

    # normalize name
    loc_norm = normalize_location(location)

    island_l, province_l = [], []
    for l in loc_norm:
        # check jika lokasi adalah Kabupaten/Kota
        if l in kab_kota_l:
            idx = loc_vocab[loc_vocab['Kabupaten/Kota']==l].index.values[0]
            island = loc_vocab.at[idx, 'Pulau']
            province = loc_vocab.at[idx, 'Provinsi']
        elif l in prov_l:
            idx = loc_vocab[loc_vocab['Provinsi']==l].index.values[0]
            island = loc_vocab.at[idx, 'Pulau']
            province = l
        else:
            island = np.nan
            province = np.nan
        island_l.append(island)
        province_l.append(province)
    return island_l, province_l

def count_job_vacancy_by_province(df):
    # tambah column count_job_vacancy_by_province
    count_job_vacancy = df.province.value_counts()

    # create dictionary
    count_job_vacancy_by_province = list()
    for idx, row in df.iterrows():
        prov = row['province']
        count_job_vacancy_by_province.append(count_job_vacancy[prov])
    return count_job_vacancy_by_province

def feature_engineering(df):
    df['island'], df['province'] = extract_location(df['company_location'])
    
    # drop col 'company_location'
    df = df.drop(['company_location'], axis=1)

    # drop rows that have island and province not identified
    df = df.dropna().reset_index(drop=True)

    # Ubah nama provinsi supaya sama dengan data Geojson
    df['province'] = df['province'].str.replace('DKI Jakarta', 'Daerah Khusus Ibukota Jakarta', regex=False)
    df['province'] = df['province'].str.replace('DI Yogyakarta', 'Daerah Istimewa Yogyakarta', regex=False)

    # add col count_job_vacancy_by_province
    df['count_job_vacancy_by_province'] = count_job_vacancy_by_province(df)

    # add id loker
    id_loker = [i+1 for i in range(len(df))]
    df['id_loker'] = id_loker

    return df

In [11]:
data_final = feature_engineering(data_preprocessed)

In [12]:
data_final.head()

,platform,job_position,company_name,salary,avg_process_time,company_size,career_level,years_of_experience,qualification,industry,employment_type,job_function,benefits,island,province,count_job_vacancy_by_province,id_loker
0,Jobstreet,Staff It,PT Nichirin Indonesia,"IDR 4,900,000 - 5,800,000",28,51 - 200 pekerja,Pegawai (non-manajemen & non-supervisor),1,Sarjana (S1),Automobil/Mesin Tambahan Automotif/Kendaraan,Penuh Waktu,"Komputer/Teknologi Informasi, IT-Admin Jaringa...",Tidak Dicantumkan,Jawa,Jawa Barat,2949,1
1,Jobstreet,Management Trainee,PT Sulfindo Adiusaha,"IDR 6,500,000 - 8,500,000",28,201 - 500 pekerja,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,0,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Bahan Kimia/Pupuk/Pestisida,Penuh Waktu,"Penjualan/Pemasaran, Pemasaran/Pengembangan Bi...",Bonus,Jawa,Banten,2000,2
2,Jobstreet,Admin,WINGS Group,"IDR 4,800,000 - 5,000,000",19,Lebih dari 5000 pekerja,Pegawai (non-manajemen & non-supervisor),1,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Produk Konsumen/Barang konsumen yang bergerak ...,Penuh Waktu,"Manufaktur, Pembelian/Manajemen Material",Makan Siang,Jawa,Daerah Khusus Ibukota Jakarta,9980,3
3,Jobstreet,Kepala Cabang,PT Puji Lestari Purnama,"IDR 9,000,000 - 15,000,000",26,1- 50 pekerja,Manajer/Asisten Manajer,5,Sarjana (S1),Umum & Grosir,Penuh Waktu,"Penjualan/Pemasaran, Pemasaran/Pengembangan Bi...",Tidak Dicantumkan,Sumatera,Lampung,65,4
4,Jobstreet,Operation Engineer,FIBERSTAR (PT. Mega Akses Persada),"IDR 4,000,000 - 7,000,000",22,201 - 500 pekerja,Pegawai (non-manajemen & non-supervisor),2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Telekomunikasi,Penuh Waktu,"Teknik, Teknik Elektronika","Bonus, Parkir, Penglihatan, Bpjs",Jawa,Daerah Khusus Ibukota Jakarta,9980,5


# Export Data

In [13]:
data = data_final.copy()

In [14]:
# export data master
data.to_csv('jobstreet_jan_feb_21.csv', index=False)

In [15]:
# export data pendidikan/qualification
df_qualification = pd.DataFrame(columns=['id_loker', 'province', 'qualification'])

for idx, row in data.iterrows():
    split_qualification = [q.strip() for q in row['qualification'].split(',')]
    for q in split_qualification:
        input_data = {'id_loker': row['id_loker'],
                      'province': row['province'],
                      'qualification': q}

        df_qualification = df_qualification.append(input_data, ignore_index=True)

In [16]:
df_qualification.head()

,id_loker,province,qualification
0,1,Jawa Barat,Sarjana (S1)
1,2,Banten,Sertifikat Professional
2,2,Banten,D3 (Diploma)
3,2,Banten,D4 (Diploma)
4,2,Banten,Sarjana (S1)


In [17]:
df_qualification.to_csv('jobstreet_qualification_jan_feb_21.csv', index=False)

In [18]:
# export count of job vacancy
df_count_job = pd.DataFrame(columns=['id_loker', 'province', 'count_job_vacancy'])

for idx, row in data.iterrows():
    input_data = {'id_loker': row['id_loker'],
                  'province': row['province'],
                  'count_job_vacancy': row['count_job_vacancy_by_province']}
    df_count_job = df_count_job.append(input_data, ignore_index=True)

In [19]:
df_count_job.head()

,id_loker,province,count_job_vacancy
0,1,Jawa Barat,2949
1,2,Banten,2000
2,3,Daerah Khusus Ibukota Jakarta,9980
3,4,Lampung,65
4,5,Daerah Khusus Ibukota Jakarta,9980


In [20]:
df_count_job.to_csv('jobstreet_count_job_jan_feb_21.csv', index=False)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20527 entries, 0 to 20526
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   platform                       20527 non-null  object
 1   job_position                   20527 non-null  object
 2   company_name                   20527 non-null  object
 3   salary                         20527 non-null  object
 4   avg_process_time               20527 non-null  object
 5   company_size                   20527 non-null  object
 6   career_level                   20527 non-null  object
 7   years_of_experience            20527 non-null  object
 8   qualification                  20527 non-null  object
 9   industry                       20527 non-null  object
 10  employment_type                20527 non-null  object
 11  job_function                   20527 non-null  object
 12  benefits                       20527 non-null  object
 13  i

In [22]:
# export employment type
df_emp_type = pd.DataFrame(columns=['id_loker', 'province', 'employment_type'])

for idx, row in data.iterrows():
    split_emp = [e.strip() for e in row['employment_type'].split(',')]
    for e in split_emp:
        input_data = {'id_loker': row['id_loker'],
                      'province': row['province'],
                      'employment_type': e}

        df_emp_type = df_emp_type.append(input_data, ignore_index=True)

In [23]:
df_emp_type.head()

,id_loker,province,employment_type
0,1,Jawa Barat,Penuh Waktu
1,2,Banten,Penuh Waktu
2,3,Daerah Khusus Ibukota Jakarta,Penuh Waktu
3,4,Lampung,Penuh Waktu
4,5,Daerah Khusus Ibukota Jakarta,Penuh Waktu


In [24]:
df_emp_type.to_csv('jobstreet_employment_type_jan_feb_21.csv', index=False)

In [25]:
# export job_function
df_job_function = pd.DataFrame(columns=['id_loker', 'province', 'job_function'])

for idx, row in data.iterrows():
    split_func = [f.strip() for f in row['job_function'].split(',')]
    for f in split_func:
        input_data = {'id_loker': row['id_loker'],
                      'province': row['province'],
                      'job_function': f}

        df_job_function = df_job_function.append(input_data, ignore_index=True)

In [26]:
df_job_function.head()

,id_loker,province,job_function
0,1,Jawa Barat,Komputer/Teknologi Informasi
1,1,Jawa Barat,IT-Admin Jaringan/Sistem/Database
2,2,Banten,Penjualan/Pemasaran
3,2,Banten,Pemasaran/Pengembangan Bisnis
4,3,Daerah Khusus Ibukota Jakarta,Manufaktur


In [27]:
df_job_function.to_csv('jobstreet_job_function_jan_feb_21.csv', index=False)

In [28]:
# export career_level
df_career_level = pd.DataFrame(columns=['id_loker', 'province', 'career_level'])

for idx, row in data.iterrows():
    input_data = {'id_loker': row['id_loker'],
                  'province': row['province'],
                  'career_level': row['career_level']}
    df_career_level = df_career_level.append(input_data, ignore_index=True)

In [29]:
df_job_function.to_csv('jobstreet_career_level_jan_feb_21.csv', index=False)